<a href="https://colab.research.google.com/github/adamstiefel/AI-Business-Agents/blob/main/Optimization_Advisor_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠💡 Optimization Advisor Agent 🚀

**Leveraging AI to Suggest Process Redesigns with Lean, Six Sigma & RPA Insights**

## What This Tool Does

This Google Colab notebook implements an **Optimization Advisor Agent**. Given a textual description of a current business process, this agent uses a Large Language Model (LLM) to:

1.  **Analyze** the provided process description.
2.  **Identify** potential areas for improvement based on:
    * **Lean Principles:** Focusing on eliminating waste (e.g., TIM WOODS: Transport, Inventory, Motion, Waiting, Overproduction, Over-processing, Defects).
    * **Six Sigma Concepts:** Highlighting opportunities to reduce variation and defects (though this agent will focus more on qualitative suggestions unless quantitative data is provided).
    * **Robotic Process Automation (RPA) Patterns:** Pinpointing repetitive, rules-based, manual tasks that are good candidates for automation.
3.  **Suggest** concrete redesign ideas and optimization strategies.

Think of this agent as an initial brainstorming partner or a virtual junior consultant that can provide a fresh perspective on your processes.

## How This Delivers Business Value (The "Makes Money" Aspect)

This agent helps organizations unlock significant value by identifying actionable improvements:

* **💰 Direct Cost Reduction:** Lean suggestions target waste elimination (materials, time, effort), while RPA suggestions can drastically cut costs associated with manual labor for repetitive tasks.
* 🚀 **Increased Throughput & Capacity:** Optimized processes can handle more work with existing resources, improving overall business capacity and revenue potential.
* ✨ **Enhanced Quality & Customer Satisfaction:** Reducing process defects and variability (Six Sigma insights) leads to better product/service quality, less rework, and happier customers.
* ⏱️ **Drastically Reduced Cycle Times:** Streamlining flow (Lean) and automating tasks (RPA) can significantly shorten the time from process start to finish.
* 🧑‍💼 **Better Resource Utilization & Employee Engagement:** Automating mundane tasks frees up employees for more strategic, engaging, and higher-value work, improving morale and productivity.
* 📈 **Data-Driven (Qualitative) Recommendations:** Provides structured suggestions based on established methodologies, offering a strong starting point for deeper analysis and business case development for changes.
* 💡 **Innovation Catalyst:** Helps uncover "hidden factory" inefficiencies and sparks ideas for transformative process improvements.

The suggestions provided will require human review, validation, and further analysis, but this agent can massively accelerate the initial stages of your process improvement journey.

In [1]:
#@title 2. Setup and Installations
# --- Install necessary libraries ---
!pip install openai -q

import openai
from IPython.display import display, Markdown # For formatted output

print("Libraries installed and imported.")

Libraries installed and imported.


In [2]:
#@title 3. OpenAI API Key Configuration
# --- Configure your OpenAI API Key ---
# Recommended: Use Colab Secrets (Key icon on the left sidebar)
# Add a secret named 'OPENAI_API_KEY' with your actual OpenAI API key.
from google.colab import userdata

try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY:
        openai.api_key = OPENAI_API_KEY
        print("✅ OpenAI API Key loaded successfully from Colab Secrets.")
    else:
        print("⚠️ OpenAI API Key not found in Colab Secrets. Please add it.")
        print("Alternatively, you can paste it directly below (less secure, not recommended for sharing):")
        # OPENAI_API_KEY_MANUAL = "" # PASTE YOUR KEY HERE IF NOT USING SECRETS
        # if OPENAI_API_KEY_MANUAL:
        #    openai.api_key = OPENAI_API_KEY_MANUAL
        #    print("✅ OpenAI API Key set directly (less secure).")
        # else:
        #    print("🛑 API Key not provided.")

except Exception as e:
    print(f"⚠️ Error accessing Colab Secrets: {e}")
    print("Please ensure you have an OpenAI API key. You might need to set it manually if Colab Secrets are unavailable in your environment.")
    OPENAI_API_KEY = None

if not openai.api_key:
    print("🛑 OpenAI API Key is NOT SET. LLM functionality will not work.")

✅ OpenAI API Key loaded successfully from Colab Secrets.


## Quick Guide to Optimization Lenses

This agent will analyze your process through these common improvement methodologies:

### Lean Principles (Focus: Eliminating Waste)
Lean aims to maximize customer value while minimizing waste. Common wastes (often remembered by the acronym **TIM WOODS (or DOWNTIME)**) include:
* **T**ransport: Unnecessary movement of materials, information, or people.
* **I**nventory: Excess products, materials, or information waiting to be processed.
* **M**otion: Unnecessary movement by people (e.g., reaching for tools, walking).
* **W**aiting: Idle time for people, equipment, or information.
* **O**verproduction: Producing more than needed, sooner than needed, or faster than needed.
* **O**ver-processing: Doing more work than necessary to meet customer requirements (e.g., excessive approvals, redundant checks).
* **D**efects: Products or services that are out of spec, requiring rework or scrapping.
* **(S)kills Unused (Non-Utilized Talent):** Failing to use the knowledge, skills, and creativity of employees.

### Six Sigma (Focus: Reducing Variation & Defects)
While this agent focuses on qualitative suggestions, Six Sigma's core idea is to reduce process variation and defects to improve quality and consistency. Key concepts include:
* **DMAIC Cycle:** Define, Measure, Analyze, Improve, Control.
* Focus on data-driven decisions to identify root causes of problems.

### Robotic Process Automation (RPA) (Focus: Automating Repetitive Tasks)
RPA is ideal for tasks that are:
* **Repetitive and High-Volume:** Performed many times.
* **Rules-Based:** Follow clear, predefined rules with few exceptions.
* **Digital Data Input:** Involve interaction with digital systems (reading from screens, typing, using mouse).
* **Prone to Human Error:** Tedious tasks where humans might make mistakes.
* **Stable Processes:** The underlying process doesn't change too frequently.

    *Examples: Data entry, data migration, form filling, report generation, reconciliations, system integrations (basic).*

In [4]:
#@title 5. Input: Describe Your Current Process
#@markdown Please provide a detailed description of the business process you want to optimize.
#@markdown The more detail you provide, the better the suggestions will be.
#@markdown Consider including:
#@markdown - The overall goal of the process.
#@markdown - Key steps in sequence.
#@markdown - Roles involved (who does what).
#@markdown - Systems or tools used.
#@markdown - Any known pain points, delays, or areas of concern.
#@markdown - Input and output of the process.
#@markdown - Any existing metrics or KPIs (if available, though not strictly required for this qualitative advisor).

process_description_input = """
Example: Customer Order Fulfillment Process

Goal: To process customer orders accurately and ship products efficiently.

Steps:
1. Order is received via online portal (System: E-commerce Platform). Data Entry Clerk manually re-enters order details into ERP.
2. Inventory Check: Warehouse staff manually checks stock levels in a spreadsheet for each item. If item out of stock, customer is emailed manually (takes 1-2 days for notification).
3. Payment Processing: Finance team receives a daily batch of orders and manually processes payments through a separate payment gateway. Reconciles with ERP at end of day.
4. Order Picking: Warehouse staff picks items based on a printed pick list.
5. Order Packing: Items are packed, and a shipping label is manually generated by copying address details into a courier's web portal.
6. Shipping: Courier picks up packed orders. Tracking number is manually emailed to customer by customer service.
7. Post-Shipment: Customer service handles any inquiries regarding order status, often by checking multiple systems.

Known Pain Points: Data entry errors from manual input, delays in out-of-stock notifications, delays in payment processing affecting picking, time spent on manual label generation and tracking updates, customer complaints about slow status updates.
""" #@param {type:"string"}

if not process_description_input.strip() or process_description_input.strip() == "Example: Customer Order Fulfillment Process...": # A more robust check for default
    print("⚠️ Please replace the example process description with your own process details above.")
else:
    print("✅ Process description received.")

✅ Process description received.


In [6]:
#@title 6. Generate Optimization Suggestions

#@markdown Select the LLM model to use (GPT-4 class models are highly recommended for this complex task):
llm_model_advisor = "gpt-4-turbo-preview" #@param ["gpt-4-turbo-preview", "gpt-4", "gpt-3.5-turbo"]
#@markdown ---
#@markdown Focus of Advice (select multiple if desired, will be combined in prompt):
focus_lean = True #@param {type:"boolean"}
focus_rpa = True #@param {type:"boolean"}
focus_six_sigma_qualitative = True #@param {type:"boolean"}


def get_optimization_suggestions(process_description, model_name="gpt-4-turbo-preview", lean=True, rpa=True, six_sigma=True):
    if not openai.api_key:
        return "Error: OpenAI API Key not set. Please configure it in Step 3."
    # A more robust check for the default/empty process description
    if not process_description or process_description.strip() == "" or \
       ("Example: Customer Order Fulfillment Process" in process_description and len(process_description) < 300): # Check if it's very close to default
        return "Error: Please provide a valid and detailed process description in Step 5 before generating suggestions."

    focus_areas = []
    if lean:
        focus_areas.append("Lean principles (especially identifying the 7 wastes - TIM WOODS: Transport, Inventory, Motion, Waiting, Overproduction, Over-processing, Defects, and opportunities for flow/pull systems)")
    if rpa:
        focus_areas.append("Robotic Process Automation (RPA) suitability (identifying repetitive, rules-based, digital tasks for automation)")
    if six_sigma:
        focus_areas.append("qualitative Six Sigma opportunities (reducing chances for errors, improving consistency, standardizing work - without requiring statistical data input here)")

    if not focus_areas:
        return "Error: Please select at least one focus area for advice (Lean, RPA, Six Sigma)."

    # Constructing the focus_prompt_segment more robustly
    if len(focus_areas) == 1:
        focus_prompt_segment = focus_areas[0]
    elif len(focus_areas) == 2:
        focus_prompt_segment = focus_areas[0] + " and " + focus_areas[1]
    else: # More than 2
        focus_prompt_segment = ", ".join(focus_areas[:-1]) + ", and " + focus_areas[-1]


    prompt = f"""
    You are an expert Process Optimization Advisor with deep knowledge in Lean methodologies, Six Sigma principles, and Robotic Process Automation (RPA).
    Your task is to analyze the following business process description and provide actionable suggestions for redesign and optimization.

    Current Process Description:
    ---
    {process_description}
    ---

    Please provide your analysis and suggestions structured as follows:
    1.  **Overall Process Summary & Key Observations:** Briefly summarize your understanding of the process and any immediate high-level observations or concerns.
    2.  **Optimization Opportunities & Suggestions (focused on {focus_prompt_segment}):**
        * For each identified opportunity, clearly state:
            * **The Issue/Opportunity:** What specific part of the process or problem are you addressing?
            * **Principle/Pattern:** Which Lean waste, RPA pattern, or Six Sigma concept does this relate to?
            * **Suggested Redesign/Action:** What specific change or action do you recommend? Explain how this would improve the process.
            * **Expected Benefits:** What are the likely positive outcomes (e.g., reduced time, lower cost, improved quality, better employee experience)?
    3.  **Potential RPA Candidates (if RPA focus is selected and applicable):** List specific tasks that seem suitable for RPA, explaining why.
    4.  **General Recommendations & Next Steps:** Any overarching advice or suggested next steps for the organization to consider.

    Be specific, practical, and explain your reasoning clearly. Assume the goal is to make the process more efficient, cost-effective, and robust.
    """

    print(f"\n🤖 Requesting optimization advice from {model_name} based on selected focus areas...")
    print("This may take a moment, especially with more advanced models and detailed descriptions...")

    try:
        response = openai.chat.completions.create(
            model=model_name, # The API call itself uses 'model'
            messages=[
                {"role": "system", "content": "You are an expert Process Optimization Advisor."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=2500
        )
        suggestions = response.choices[0].message.content
        print("✅ Optimization suggestions received.")
        return suggestions
    except openai.APIError as e: # Specific OpenAI errors
        return f"OpenAI API Error: {e}"
    except Exception as e: # Other potential errors
        return f"An unexpected error occurred: {e}"

# --- Main execution for this cell ---
# More robust check for empty/default process_description_input
is_default_description = "Example: Customer Order Fulfillment Process" in process_description_input and len(process_description_input) < (len("Example: Customer Order Fulfillment Process...") + 100) # Check if it's very similar to default

if 'process_description_input' in globals() and process_description_input.strip() and not is_default_description:
    if openai.api_key: # Check if API key is actually set
        optimization_advice = get_optimization_suggestions(
            process_description_input,
            model_name=llm_model_advisor,  # <-- CORRECTED HERE
            lean=focus_lean,
            rpa=focus_rpa,
            six_sigma=focus_six_sigma_qualitative
        )
        if optimization_advice.startswith("Error:"):
            print(f"\n⚠️ {optimization_advice}")
            display(Markdown(f"**Error:** {optimization_advice}")) # Display error in markdown too
        else:
            print("\n--- Optimization Advisor's Report ---")
            display(Markdown(optimization_advice))
    else:
        error_message = "🛑 OpenAI API Key not configured. Please set it up in Step 3."
        print(f"\n{error_message}")
        display(Markdown(f"**{error_message}**"))
else:
    error_message = "⚠️ Please provide a detailed and unique process description in Step 5 before generating suggestions. The current input might be empty or too similar to the example."
    print(f"\n{error_message}")
    display(Markdown(f"**{error_message}**"))


🤖 Requesting optimization advice from gpt-4-turbo-preview based on selected focus areas...
This may take a moment, especially with more advanced models and detailed descriptions...
✅ Optimization suggestions received.

--- Optimization Advisor's Report ---


### 1. **Overall Process Summary & Key Observations:**

The customer order fulfillment process involves several manual steps from order receipt to shipping, including manual data entry, inventory checks, payment processing, and shipping label generation. Key observations highlight significant manual intervention leading to data entry errors, delays in notifying customers about stock levels, inefficiencies in payment processing, and overall delays in order fulfillment. These manual processes contribute to increased lead times, potential for errors, and customer dissatisfaction.

### 2. **Optimization Opportunities & Suggestions:**

#### **Order Entry & ERP Integration:**
- **The Issue/Opportunity:** Manual re-entry of order details into the ERP system.
- **Principle/Pattern:** Defects, Over-processing.
- **Suggested Redesign/Action:** Implement an integration between the e-commerce platform and the ERP system to automate order entry. Use API or middleware solutions for real-time data synchronization.
- **Expected Benefits:** Reduces data entry errors, accelerates order processing, and improves accuracy.

#### **Inventory Check Automation:**
- **The Issue/Opportunity:** Manual inventory checks and customer notifications.
- **Principle/Pattern:** Waiting, Motion.
- **Suggested Redesign/Action:** Automate inventory checks using the ERP system. Implement an automated notification system for out-of-stock items directly from the ERP.
- **Expected Benefits:** Decreases waiting time for inventory checks, speeds up customer notifications, and reduces manual labor.

#### **Payment Processing Efficiency:**
- **The Issue/Opportunity:** Batch processing of payments and manual reconciliation.
- **Principle/Pattern:** Waiting, Over-processing.
- **Suggested Redesign/Action:** Automate payment processing with real-time integration between the e-commerce platform and payment gateway. Automate reconciliation in the ERP system.
- **Expected Benefits:** Reduces delays in order processing, minimizes manual reconciliation efforts, and accelerates cash flow.

#### **Digital Pick Lists & Packing Optimization:**
- **The Issue/Opportunity:** Use of printed pick lists and manual shipping label generation.
- **Principle/Pattern:** Motion, Over-processing.
- **Suggested Redesign/Action:** Implement digital pick lists accessible via tablets or handheld devices. Automate shipping label generation directly from the ERP system.
- **Expected Benefits:** Reduces paper usage, minimizes walking and searching time, and speeds up packing and shipping processes.

### 3. **Potential RPA Candidates:**

- **Order Details to ERP Entry:** Suitable due to its repetitive, rule-based nature and digital format.
- **Automated Inventory Level Checks:** Ideal for RPA as it involves querying a database and applying rules to determine stock levels.
- **Payment Processing & Reconciliation:** These tasks are repetitive, rule-based, and involve digital data, making them perfect for RPA.
- **Shipping Label Generation:** Involves accessing customer address details and generating labels, a repetitive and rules-based task suitable for RPA.

### 4. **General Recommendations & Next Steps:**

- **Conduct a Process Mapping Session:** To fully understand current state processes and identify all inefficiencies not immediately apparent.
- **Prioritize Quick Wins:** Focus initially on implementing RPA for high-volume, repetitive tasks such as data entry and payment processing.
- **Invest in Integration Technologies:** To facilitate seamless data flow between the e-commerce platform, ERP, and payment systems.
- **Lean Training for Staff:** Equip employees with Lean thinking principles to continuously identify and eliminate waste in their daily tasks.
- **Monitor and Adjust:** Implement KPIs to monitor the impact of changes and be prepared to iterate based on results.

By addressing these areas, the organization can expect to see reduced processing times, lower operational costs, fewer errors, and improved customer satisfaction. The focus should be on creating a streamlined, automated process that reduces manual intervention and leverages technology to enhance efficiency and accuracy.